# Deploy MLflow model to online endpoints with a custom environment and scoring script

Learn how to deploy your [MLflow](https://www.mlflow.org/) model to an [online endpoint](https://docs.microsoft.com/azure/machine-learning/concept-endpoints) using a custom environment and scoring script.

### Requirements - In order to benefit from this tutorial, you will need:
- This sample notebook assumes you're using online endpoints; for more information, see [What are Azure Machine Learning endpoints?](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../jobs/configuration.ipynb)
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

# 1. Connect to Azure Machine Learning Workspace
The [workspace](https://docs.microsoft.com/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

## 1.2 Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2. Create Online Endpoint

Online endpoints are endpoints that are used for online (real-time) inferencing. Online endpoints contain deployments that are ready to receive data from clients and can send responses back in real time.

To create an online endpoint we will use `ManagedOnlineEndpoint`. This class allows user to configure the following key aspects:

- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Key-based authentication and Azure ML token-based authentication are supported. Key-based authentication doesn't expire but Azure ML token-based authentication does. Possible values are `key` or `aml_token`.
- `identity`- The managed identity configuration for accessing Azure resources for endpoint provisioning and inference.
    - `type`- The type of managed identity. Azure Machine Learning supports `system_assigned` or `user_assigned identity`.
    - `user_assigned_identities` - List (array) of fully qualified resource IDs of the user-assigned identities. This property is required is `identity.type` is user_assigned.
- `description`- Description of the endpoint.

## 2.1 Configure the endpoint

In [ ]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = "diabetes-endpoint-" + endpoint_suffix

print(f"Endpoint name: {endpoint_name}")

In [ ]:
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="An online endpoint serving an MLflow model for the diabetes classification task",
    auth_mode="key",
    tags={"foo": "bar"},
)

## 2.2 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 3. Create a blue deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class. This class allows user to configure the following key aspects.

- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment where the model will run.
- `code_configuration` - The scoring script used to serve the model.
- `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](https://docs.microsoft.com/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list).
- `instance_count` - The number of instances to use for the deployment

## 3.1 Configure the deployment

Registering the model:

In [ ]:
model_name = "sklearn-diabetes"

model = Model(
    name=model_name,
    path="sklearn-diabetes/model",
    type=AssetTypes.MLFLOW_MODEL,
    description="MLflow model for the diabetes classification problem",
)

Creating an environment to perform inference:

In [ ]:
environment = Environment(
    conda_file="sklearn-diabetes/environment/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)

Creating the deployment:

In [ ]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="sklearn-diabetes/src", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

## 3.2 Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

In [ ]:
endpoint.traffic = {blue_deployment.name: 100}
ml_client.begin_create_or_update(endpoint).result()

# 4. Test the deployment

Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the invoke command with the following parameters:

- `endpoint_name` - Name of the endpoint
- `request_file` - File with request data
- `deployment_name` - Name of the specific deployment to test in an endpoint

We will send a sample request using a [sample-request-sklearn-custom.json](sample-request-sklearn-custom.json) file.

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-request-sklearn.json",
)

# 5. Get endpoint details

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

# 6. Delete the deployment and endopoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)